# LR 模型的简单实现

In [37]:
import numpy as np
import scipy 
import random
from sklearn import cross_validation,metrics
from sklearn.metrics import *

In [156]:
class LR():
    def __init__(self, lr = 0.01, L1=None, L2=None):
        self.W = None
        self.b = None
        self.epsilon = 0.001
        self.L1_W = L1
        self.L2_W = L2
        self.X = None
        self.y = None
        self.y_hat = None
        self.lr = lr
        self.W_gred = None
        
    def sigmoid(self, X):
        return 1/(1 + np.exp(-X))
    
    
    def get_coef(self):
        return self.W, self.b
    
    def _get_gred(self):
        m  = self.X.shape[0]
        y_diff =  self.y_hat - self.y 
        W_gred = np.sum(y_diff*self.X, axis=0,keepdims=True)  / m
        W_gred = W_gred.T

        b_gred = np.sum(y_diff) / m
        self.W_gred = W_gred
        return W_gred, b_gred
    
    def _init_param(self):
        self.W = np.random.random((self.X.shape[1],1)) 
        self.b = 0
    
    def predict_prob(self,X):
        wx_b = np.dot(X , self.W) + self.b
        return self.sigmoid(wx_b).reshape(X.shape[0])
    
    def fit(self, X, y):
        # 初始化参数
        self.X = X
        self.y = y
        self.y = self.y.reshape(y.shape[0],1)
        self._init_param()
        
        wx_b = np.dot(self.X , self.W) + self.b # n*m m*1
        self.y_hat = self.sigmoid(wx_b)
        
        W_gred, b_gred = self._get_gred()
        i = 0
        #while(max(abs(W_gred)) > self.epsilon): # 迭代
        while i < 1000:

            if self.L2_W != None:
                self.W = self.W  - self.lr * W_gred - self.L2_W*self.W
                self.b = self.b - self.lr * b_gred 
            else:                  
                self.W = self.W  - self.lr * W_gred
                self.b = self.b - self.lr * b_gred
            
            wx_b = np.dot(self.X , self.W) + self.b # n*m m*1

            self.y_hat = self.sigmoid(wx_b)
            
            W_gred, b_gred = self._get_gred()
            i += 1
        print(i)

In [157]:
#mlr = LR(L2=0.01)
mlr = LR()

mlr.fit(X_train, y_train)

1000


In [158]:
mlr.get_coef()

(array([[-0.71424969],
        [-0.71515877],
        [ 1.15594849],
        [ 0.89451808]]), -0.3545782218028741)

In [159]:
mlr.W_gred

array([[ 0.03895946],
       [ 0.02847051],
       [-0.05544449],
       [-0.04478569]])

In [160]:
m_y_test_hat = mlr.predict_prob(X_test)
metrics.roc_auc_score(y_test, m_y_test_hat)

0.9919786096256684

In [161]:
print([round(x,3) for x in m_y_test_hat.tolist()])

[0.824, 0.333, 0.004, 0.777, 0.012, 0.876, 0.009, 0.358, 0.398, 0.283, 0.76, 0.338, 0.448, 0.416, 0.475, 0.014, 0.457, 0.504, 0.017, 0.006, 0.75, 0.511, 0.021, 0.022, 0.592, 0.008, 0.015, 0.327, 0.297, 0.016, 0.695, 0.547, 0.009, 0.603, 0.82, 0.441, 0.007, 0.68, 0.373, 0.341, 0.707, 0.013, 0.611, 0.017, 0.007]


In [162]:
y_test

array([1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0])

In [163]:
m_y_test_hat

array([0.82382927, 0.33289649, 0.00411339, 0.77730794, 0.01160622,
       0.87627417, 0.00939821, 0.35840163, 0.39818279, 0.28347719,
       0.76037249, 0.33833139, 0.44829394, 0.41568667, 0.47497629,
       0.01409112, 0.45748008, 0.50430758, 0.01726311, 0.00582297,
       0.74957797, 0.5109214 , 0.02105555, 0.02244458, 0.59225962,
       0.00754089, 0.01535643, 0.32663708, 0.29701691, 0.01551912,
       0.69531379, 0.54650085, 0.00938786, 0.60283758, 0.8195099 ,
       0.44051287, 0.00731899, 0.68038925, 0.37346814, 0.34086911,
       0.70666724, 0.01314576, 0.61131322, 0.01724811, 0.00707008])

## sklearn-标准的结果

In [111]:
from sklearn.datasets import load_iris
iris=load_iris()
X=iris.data
y=iris.target
y = y//2
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0) 
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='sag')
lr.fit(X_train, y_train)


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False)

In [112]:
y_test_hat = lr.predict_proba(X_test) # 查看第一个测试样本属于各个类别的概率
y_test_hat = y_test_hat[:,1]

In [113]:
y_test

array([1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0])

In [114]:
y_test_hat

array([9.46564432e-01, 9.96477557e-02, 1.10835263e-05, 9.17665359e-01,
       6.39082963e-05, 9.82693473e-01, 4.33688195e-05, 1.95256521e-01,
       2.30978002e-01, 8.60580667e-02, 8.69767874e-01, 1.69843454e-01,
       2.94948707e-01, 2.54720231e-01, 3.59034136e-01, 8.19334501e-05,
       3.25934208e-01, 3.63259903e-01, 1.21478600e-04, 2.01732111e-05,
       8.67537416e-01, 4.31288501e-01, 2.21056829e-04, 1.92005563e-04,
       6.02259361e-01, 2.70839428e-05, 1.43802497e-04, 1.32416216e-01,
       6.84486265e-02, 1.20666769e-04, 8.20472093e-01, 4.99920160e-01,
       4.37580439e-05, 6.43182659e-01, 9.45688064e-01, 2.57121926e-01,
       3.18196313e-05, 7.54524891e-01, 1.95501761e-01, 1.25591933e-01,
       8.40392912e-01, 7.32928941e-05, 7.00582945e-01, 1.61564141e-04,
       2.70417261e-05])

In [119]:
metrics.roc_auc_score(y_test, y_test_hat)

0.9919786096256684

In [120]:
lr.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'sag',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [116]:
lr.coef_

array([[-1.38138337, -1.02517207,  2.41212782,  1.97213783]])

In [121]:
lr.intercept_

array([-3.27811487])